In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import utils
utils.set_pretty_prints()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [8]:
import torch as th
th.__version__

'1.3.1'

In [9]:
from torchvision.datasets import MNIST

In [10]:
mnist_data_train = MNIST('./datasets', download=True, train=True)

0it [00:00, ?it/s]

9920512it [00:06, 1603487.95it/s]                             


Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 131431.61it/s]           


...

1654784it [00:01, 1389613.09it/s]                             


Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw


8192it [00:00, 69628.12it/s]

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw


Processing...
Done!


In [11]:
type(mnist_data_train)

torchvision.datasets.mnist.MNIST

In [12]:
len(mnist_data_train)

60000

In [13]:
mnist_data_train[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7FCB6A181DA0>, 5)

In [14]:
mnist_data_train[0][0]

In [29]:
mnist_test_data = MNIST('./datasets', train=False, transform=lambda x: (np.array(x)/255.).astype('float32'))

In [30]:
len(mnist_test_data)

10000

In [31]:
mnist_test_data[100][0]

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.5 , 0.87, 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.25, 0.9 , 0.86, 0.41, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.05,
        0.92, 0.55, 0.02, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0. 

In [21]:
mnist_test_data[100][1]

6

In [22]:
th.cuda.is_available()

False

In [23]:
dev = th.device('cuda' if th.cuda.is_available() else 'cpu')

In [26]:
th_x_train = th.tensor([(np.array(x[0])/255).astype('float32') for x in mnist_data_train])
th_y_train = th.tensor([x[1] for x in mnist_data_train])

In [32]:
th_x_train = th_x_train.to(dev)
th_y_train = th_y_train.to(dev)

In [33]:
th_x_test = th.tensor([x[0] for x in mnist_test_data])

In [87]:
np_y_test = np.array([x[1] for x in mnist_test_data])

In [34]:
th_x_test.shape

torch.Size([10000, 28, 28])

In [35]:
th_x_train.shape

torch.Size([60000, 28, 28])

In [37]:
# tf uses BHWC
# th uses BCHW
th_x_train = th_x_train.unsqueeze(1)
th_x_train.shape

torch.Size([60000, 1, 28, 28])

In [38]:
th_x_test = th_x_test.unsqueeze(1)
th_x_test.shape

torch.Size([10000, 1, 28, 28])

In [39]:
batch_size = 128

In [40]:
th_dataloader = th.utils.data.DataLoader(
    dataset=th.utils.data.TensorDataset(th_x_train, th_y_train),
    batch_size = batch_size,
    shuffle=True,
)

In [41]:
for i, batch in enumerate(th_dataloader):
    th_x_batch, th_y_batch = batch
    break

th_x_batch.shape

torch.Size([128, 1, 28, 28])

In [42]:
th_y_batch.shape

torch.Size([128])

In [43]:
th_x_batch[0,0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [64]:
class GlobalMaxPooling2D(th.nn.Module):
    def forward(self, th_x):
        pool_window = th_x.shape[2], th_x.shape[3]
        th_pooled = th.nn.functional.max_pool2d(th_x, kernel_size=pool_window)
        # B, C, 1, 1
        th_squeezed = th.squeeze(th_pooled)
        return th_squeezed
    
class Flatten(th.nn.Module):
    def forward(self, th_x):
        return th.flatten(th_x, start_dim=1)

class MNISTClassifier(th.nn.Module):
    def __init__(self, input_shape=(1, 28, 28), filters=[32, 64], kernels=[5,3], strides=[2,1], dropout=0.5, activation=th.nn.ReLU, gmp=True):
        super().__init__()
        self.layers = th.nn.ModuleList()
        in_channels = input_shape[0]
        in_h = input_shape[1]
        for i, flt in enumerate(filters):
            kern = kernels[i]
            stride = strides[i]
            cnv_layer = th.nn.Conv2d(in_channels, flt, kernel_size=kern, stride=stride)
            self.layers.append(cnv_layer)
            in_channels = flt
            self.layers.append(activation())
            self.layers.append(th.nn.Dropout(dropout))
            out_h = int((in_h - kern) / stride + 1)
            in_h = out_h
            print("Layer {}: out volume is {}".format(i + 1, (flt, out_h, out_h)))
        if gmp:
            self.flat_layer = GlobalMaxPooling2D()
            self.out_layer = th.nn.Linear(filters[-1], 10)
        else:
            self.flat_layer = Flatten()
            self.out_layer = th.nn.Linear(out_h * out_h * filters[-1], 10)
        return
            
        
    def forward(self, th_x):
        for layer in self.layers:
            th_x = layer(th_x)
        th_x = self.flat_layer(th_x)
        th_x = self.out_layer(th_x)
        return th_x
        

In [65]:
gmp = GlobalMaxPooling2D()

In [66]:
t1 = th.tensor(np.arange(24).reshape(2,3,2,2).astype('float32'))
t1

tensor([[[[ 0.,  1.],
          [ 2.,  3.]],

         [[ 4.,  5.],
          [ 6.,  7.]],

         [[ 8.,  9.],
          [10., 11.]]],


        [[[12., 13.],
          [14., 15.]],

         [[16., 17.],
          [18., 19.]],

         [[20., 21.],
          [22., 23.]]]])

In [67]:
t1.shape

torch.Size([2, 3, 2, 2])

In [68]:
gmp(t1)

tensor([[ 3.,  7., 11.],
        [15., 19., 23.]])

In [69]:
gmp(t1).shape

torch.Size([2, 3])

In [70]:
th.flatten(t1, start_dim=1)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
        [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.]])

In [71]:
th.flatten(t1, start_dim=1).shape

torch.Size([2, 12])

In [92]:
model = MNISTClassifier(filters=[16, 32, 64], kernels=[5, 5, 3], strides=[2,2,1], gmp=False).to(dev)

Layer 1: out volume is (16, 12, 12)
Layer 2: out volume is (32, 4, 4)
Layer 3: out volume is (64, 2, 2)


In [93]:
model

MNISTClassifier(
  (layers): ModuleList(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
  )
  (flat_layer): Flatten()
  (out_layer): Linear(in_features=256, out_features=10, bias=True)
)

In [94]:
np_y_test.shape

(10000,)

In [95]:
epochs = 10
def predict(model, th_x):
    model.eval()
    with th.no_grad():
        th_yh = model(th_x)
        np_yh = th_yh.cpu().numpy()
    model.train()
    return np_yh

In [96]:
loss_fn = th.nn.CrossEntropyLoss()
opt = th.optim.Adam(
    params=model.parameters(),
    lr=0.001,
)
nr_batches = th_x_train.shape[0] // batch_size
for epoch in range(1, epochs +1):
    epoch_losses = []
    for i, batch in enumerate(th_dataloader):
        th_x_batch, th_y_batch = batch
        th_yh = model(th_x_batch)
        th_loss = loss_fn(input=th_yh, target=th_y_batch)
        opt.zero_grad()
        th_loss.backward()
        opt.step()
        epoch_losses.append(th_loss.detach().cpu().item())
        if (i % 20) == 0:
            print("\rEpoch {} {:.1f}%, loss = {:.3f}".format(epoch, i/nr_batches*100, np.mean(epoch_losses)), end='', flush=True)
    np_y = predict(model, th_x_test)
    np_preds = np_y.argmax(axis=1)
    acc = (np_preds == np_y_test).sum() / np_y_test.shape[0]
    print("\rEpoch {} loss = {:.3f}. Test acc: {:.2f}".format(epoch,  np.mean(epoch_losses), acc * 100))
        
        
        

Epoch 1 loss = 0.692. Test acc: 94.85
Epoch 2 loss = 0.302. Test acc: 96.64
Epoch 3 loss = 0.243. Test acc: 97.25
Epoch 4 loss = 0.209. Test acc: 97.63
Epoch 5 loss = 0.190. Test acc: 97.77
Epoch 6 loss = 0.178. Test acc: 98.06
Epoch 7 72.6%, loss = 0.167

KeyboardInterrupt: 

In [80]:
th_y_batch.shape

torch.Size([128])

In [81]:
th_y_batch

tensor([6, 1, 0, 2, 0, 6, 9, 1, 3, 1, 8, 9, 3, 7, 4, 4, 6, 4, 3, 4, 7, 5, 0, 4,
        0, 3, 3, 9, 4, 1, 3, 5, 3, 1, 0, 3, 9, 0, 7, 8, 8, 0, 0, 7, 2, 2, 0, 0,
        9, 8, 8, 4, 2, 7, 6, 4, 7, 6, 5, 4, 4, 3, 8, 1, 0, 3, 5, 3, 3, 2, 8, 7,
        5, 2, 3, 4, 7, 2, 7, 5, 4, 9, 2, 7, 9, 1, 3, 2, 3, 1, 2, 4, 2, 9, 8, 7,
        8, 2, 0, 8, 6, 9, 8, 4, 2, 5, 8, 7, 6, 8, 7, 9, 7, 3, 5, 3, 6, 3, 1, 3,
        6, 7, 9, 6, 6, 1, 3, 4])

In [82]:
th_yh.shape

torch.Size([128, 10])

In [83]:
th_yh[:10]

tensor([[-0.0585, -0.0776, -0.0849, -0.1194, -0.0784, -0.1397,  0.0788,  0.0007,
         -0.0019, -0.0116],
        [-0.2173, -0.0008, -0.0965, -0.0705, -0.0226, -0.0871,  0.1068,  0.0442,
          0.0141, -0.0362],
        [-0.1127,  0.0579, -0.0075, -0.1578,  0.0351, -0.0378,  0.1197,  0.0020,
          0.0859,  0.0515],
        [-0.0501, -0.0377, -0.0439, -0.1291,  0.0311, -0.1141,  0.0307,  0.0009,
          0.0478,  0.0139],
        [-0.0842, -0.0735, -0.0645, -0.1466,  0.0540, -0.0811, -0.0077,  0.0766,
          0.0062, -0.1257],
        [-0.0791,  0.0244, -0.0250, -0.0552, -0.1483, -0.1737,  0.1183,  0.0969,
          0.0667, -0.0797],
        [-0.0288, -0.0864,  0.0107, -0.1790, -0.0737, -0.0451, -0.0009, -0.0105,
         -0.0233,  0.0041],
        [ 0.0306,  0.0526, -0.0656, -0.0387, -0.0117, -0.1367,  0.0675, -0.0209,
          0.1009, -0.0019],
        [-0.1094,  0.0195, -0.0487, -0.1390, -0.0464, -0.1406,  0.0937,  0.0649,
          0.0520, -0.0706],
        [-0.0206, -

In [85]:
th.nn.functional.softmax(th_yh[:10])

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


tensor([[0.0989, 0.0970, 0.0963, 0.0930, 0.0969, 0.0912, 0.1134, 0.1049, 0.1046,
         0.1036],
        [0.0832, 0.1033, 0.0939, 0.0963, 0.1011, 0.0947, 0.1150, 0.1080, 0.1048,
         0.0997],
        [0.0887, 0.1052, 0.0986, 0.0848, 0.1029, 0.0956, 0.1119, 0.0995, 0.1082,
         0.1046],
        [0.0974, 0.0986, 0.0980, 0.0900, 0.1056, 0.0913, 0.1056, 0.1025, 0.1074,
         0.1038],
        [0.0959, 0.0969, 0.0978, 0.0901, 0.1101, 0.0962, 0.1035, 0.1126, 0.1050,
         0.0920],
        [0.0944, 0.1046, 0.0996, 0.0966, 0.0880, 0.0858, 0.1149, 0.1125, 0.1092,
         0.0943],
        [0.1013, 0.0956, 0.1054, 0.0872, 0.0969, 0.0997, 0.1042, 0.1032, 0.1019,
         0.1047],
        [0.1031, 0.1054, 0.0937, 0.0962, 0.0989, 0.0872, 0.1070, 0.0980, 0.1106,
         0.0998],
        [0.0923, 0.1050, 0.0981, 0.0896, 0.0983, 0.0895, 0.1131, 0.1099, 0.1085,
         0.0959],
        [0.1004, 0.1008, 0.0932, 0.1016, 0.0869, 0.1059, 0.1024, 0.1044, 0.1059,
         0.0985]], grad_fn=<